In [1]:
%%capture
!pip install --upgrade -tensorflow_hub
# !pip install -U -huggingface_hub

import textattack
import transformers
import torch
import time
from datasets import Dataset
import sys
import hashlib
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForMaskedLM, pipeline
from textattack.attack_recipes import (
    TextBuggerLi2018, DeepWordBugGao2018, TextFoolerJin2019, BERTAttackLi2020
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.models.wrappers import ModelWrapper

sys.path.append('../')
from eval_utils import *
sys.path.pop()

2023-08-05 17:23:00.332035: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-05 17:23:01.357230: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# set a seed, because reproducability is cool
np.random.seed(int(hashlib.sha256('Harrison Gietz'.encode('utf-8')).hexdigest(), 16) % 2**32)
torch.cuda.empty_cache()

device = input('enter a device name to run on: ')
dataset_val = input('Enter the number of samples to run on (100 or 1000): ')
defense = input('Specify a defense type among "default", "logit", "maj_log", "one_hot": ')

ag_tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-ag-news")
ag_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
ag_model.to(device)
ag_pipeline = pipeline('sentiment-analysis', model=ag_model, tokenizer=ag_tokenizer)
ag_pipeline.device = next(ag_model.parameters()).device

ag_model_directory = "../../../models/bert-uncased_maskedlm_agnews_july31" #first diff
finetuned_ag_maskedlm = BertForMaskedLM.from_pretrained(ag_model_directory)
finetuned_ag_maskedlm.to(device)
ag_fill_mask = pipeline("fill-mask", model=finetuned_ag_maskedlm, tokenizer=ag_tokenizer)
ag_fill_mask.device = next(ag_model.parameters()).device

num_voter = 11
mask_pct = 0.3    
    
attack = DeepWordBugGao2018

if dataset_val == '100':
    loaded_ag_100 = Dataset.load_from_disk('../data/filtered_ag_clean_100')
    ag_100 = textattack.datasets.Dataset(convert_to_tuples(loaded_ag_100))
    dataset = ag_100
    dataset_name = 'ag-news100'
elif dataset_val =='1000':
    loaded_ag_1000 = Dataset.load_from_disk('../data/filtered_ag_clean_1000')
    ag_1000 = textattack.datasets.Dataset(convert_to_tuples(loaded_ag_1000))
    dataset = ag_1000
    dataset_name = 'ag-news1000'
else:
    raise ValueError('Number of samples not supported')
    
if defense == "default":
    ag_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(ag_model, ag_tokenizer)
    print(ag_wrapper)
elif defense == "logit":
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'logit')
elif defense == 'maj_log':
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'maj_log')
elif defense == "one_hot":
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'maj_one_hot')
else:
    raise ValueError('Not a valid defense type.')
    
print(f'using num_voter = {num_voter} and mask_pct = {mask_pct} with dataset = {dataset_name}...')

# Parse the attack name
attack_name = parse_attack_name(attack)
attack = attack.build(ag_wrapper)

# Set up arguments for the attack
attack_args = textattack.AttackArgs(
    num_examples=len(dataset),
    log_to_csv=f'{attack_name}_{dataset_name}_mp{mask_pct}_nv{num_voter}_{defense}_log.csv',
    checkpoint_interval=25, 
    checkpoint_dir="chkpts_2", 
    disable_stdout=True
)
# Perform the attack and save the results
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

print(f'The above are results for {attack_name}_{dataset_name}_mp{mask_pct}_nv{num_voter}_{defense}.')

enter a device name to run on: cuda:1
Enter the number of samples to run on (100 or 1000): 1000
Specify a defense type among "default", "logit", "maj_log", "one_hot": logit
using num_voter = 11 and mask_pct = 0.3 with dataset = ag-news1000...
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 




  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(

  0%|          | 1/1000 [00:35<9:58:33, 35.95s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/1000 [00:36<10:05:20, 36.36s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 2/1000 [00:47<6:35:20, 23.77s/it] 
[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   0%|          | 2/1000 [00:47<6:35:37, 23.78s/it]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   0%|          | 3/1000 [01:44<9:41:14, 34.98s/it]
[Succeeded / Failed / Skipped / Total] 2 / 1 / 0 / 3:   0%|          | 3/1000 [01:44<9:41:21, 34.99s/it]
[Succeeded / Failed / Skipped / Total] 2 / 1 / 0 / 3:   0%|          | 4/1000 [02:20<9:43:58, 35.18s/it]
[Succeeded / Failed / Skipped / Total] 2 / 2 / 0 / 4:   0%| 


[Succeeded / Failed / Skipped / Total] 8 / 16 / 1 / 25:   3%|▎         | 26/1000 [20:52<13:01:52, 48.16s/it]
[Succeeded / Failed / Skipped / Total] 8 / 17 / 1 / 26:   3%|▎         | 26/1000 [20:52<13:01:52, 48.17s/it]
[Succeeded / Failed / Skipped / Total] 8 / 17 / 1 / 26:   3%|▎         | 27/1000 [21:18<12:48:00, 47.36s/it]
[Succeeded / Failed / Skipped / Total] 9 / 17 / 1 / 27:   3%|▎         | 27/1000 [21:18<12:48:01, 47.36s/it]
[Succeeded / Failed / Skipped / Total] 9 / 17 / 1 / 27:   3%|▎         | 28/1000 [22:07<12:47:58, 47.41s/it]
[Succeeded / Failed / Skipped / Total] 9 / 18 / 1 / 28:   3%|▎         | 28/1000 [22:07<12:47:59, 47.41s/it]
[Succeeded / Failed / Skipped / Total] 9 / 18 / 1 / 28:   3%|▎         | 29/1000 [22:07<12:20:59, 45.79s/it]
[Succeeded / Failed / Skipped / Total] 9 / 18 / 2 / 29:   3%|▎         | 29/1000 [22:07<12:21:00, 45.79s/it]
[Succeeded / Failed / Skipped / Total] 9 / 18 / 2 / 29:   3%|▎         | 30/1000 [22:48<12:17:34, 45.62s/it]
[Succeeded / Faile


[Succeeded / Failed / Skipped / Total] 13 / 34 / 3 / 50:   5%|▌         | 51/1000 [40:07<12:26:44, 47.21s/it]
[Succeeded / Failed / Skipped / Total] 13 / 35 / 3 / 51:   5%|▌         | 51/1000 [40:07<12:26:44, 47.21s/it]
[Succeeded / Failed / Skipped / Total] 13 / 35 / 3 / 51:   5%|▌         | 52/1000 [41:14<12:31:51, 47.59s/it]
[Succeeded / Failed / Skipped / Total] 13 / 36 / 3 / 52:   5%|▌         | 52/1000 [41:14<12:31:52, 47.59s/it]
[Succeeded / Failed / Skipped / Total] 13 / 36 / 3 / 52:   5%|▌         | 53/1000 [42:08<12:33:02, 47.71s/it]
[Succeeded / Failed / Skipped / Total] 13 / 37 / 3 / 53:   5%|▌         | 53/1000 [42:08<12:33:02, 47.71s/it]
[Succeeded / Failed / Skipped / Total] 13 / 37 / 3 / 53:   5%|▌         | 54/1000 [42:23<12:22:42, 47.11s/it]
[Succeeded / Failed / Skipped / Total] 14 / 37 / 3 / 54:   5%|▌         | 54/1000 [42:23<12:22:42, 47.11s/it]
[Succeeded / Failed / Skipped / Total] 14 / 37 / 3 / 54:   6%|▌         | 55/1000 [43:27<12:26:46, 47.41s/it]
[Succeede


[Succeeded / Failed / Skipped / Total] 22 / 48 / 5 / 75:   8%|▊         | 76/1000 [57:35<11:40:11, 45.47s/it]
[Succeeded / Failed / Skipped / Total] 22 / 49 / 5 / 76:   8%|▊         | 76/1000 [57:35<11:40:12, 45.47s/it]
[Succeeded / Failed / Skipped / Total] 22 / 49 / 5 / 76:   8%|▊         | 77/1000 [58:41<11:43:37, 45.74s/it]
[Succeeded / Failed / Skipped / Total] 22 / 50 / 5 / 77:   8%|▊         | 77/1000 [58:41<11:43:38, 45.74s/it]
[Succeeded / Failed / Skipped / Total] 22 / 50 / 5 / 77:   8%|▊         | 78/1000 [59:44<11:46:16, 45.96s/it]
[Succeeded / Failed / Skipped / Total] 23 / 50 / 5 / 78:   8%|▊         | 78/1000 [59:45<11:46:16, 45.96s/it]
[Succeeded / Failed / Skipped / Total] 23 / 50 / 5 / 78:   8%|▊         | 79/1000 [1:00:30<11:45:30, 45.96s/it]
[Succeeded / Failed / Skipped / Total] 23 / 51 / 5 / 79:   8%|▊         | 79/1000 [1:00:30<11:45:30, 45.96s/it]
[Succeeded / Failed / Skipped / Total] 23 / 51 / 5 / 79:   8%|▊         | 80/1000 [1:01:32<11:47:38, 46.15s/it]
[Su

...


[Succeeded / Failed / Skipped / Total] 257 / 634 / 34 / 925:  93%|█████████▎| 926/1000 [15:50:22<1:15:56, 61.58s/it]
[Succeeded / Failed / Skipped / Total] 258 / 634 / 34 / 926:  93%|█████████▎| 926/1000 [15:50:22<1:15:56, 61.58s/it]
[Succeeded / Failed / Skipped / Total] 258 / 634 / 34 / 926:  93%|█████████▎| 927/1000 [15:51:40<1:14:56, 61.60s/it]
[Succeeded / Failed / Skipped / Total] 258 / 635 / 34 / 927:  93%|█████████▎| 927/1000 [15:51:40<1:14:56, 61.60s/it]
[Succeeded / Failed / Skipped / Total] 258 / 635 / 34 / 927:  93%|█████████▎| 928/1000 [15:53:01<1:13:56, 61.62s/it]
[Succeeded / Failed / Skipped / Total] 258 / 636 / 34 / 928:  93%|█████████▎| 928/1000 [15:53:01<1:13:56, 61.62s/it]
[Succeeded / Failed / Skipped / Total] 258 / 636 / 34 / 928:  93%|█████████▎| 929/1000 [15:54:08<1:12:55, 61.62s/it]
[Succeeded / Failed / Skipped / Total] 258 / 637 / 34 / 929:  93%|█████████▎| 929/1000 [15:54:08<1:12:55, 61.62s/it]
[Succeeded / Failed / Skipped / Total] 258 / 637 / 34 / 929:  9


[Succeeded / Failed / Skipped / Total] 262 / 653 / 35 / 950:  95%|█████████▌| 951/1000 [16:19:15<50:27, 61.78s/it]
[Succeeded / Failed / Skipped / Total] 262 / 654 / 35 / 951:  95%|█████████▌| 951/1000 [16:19:15<50:27, 61.78s/it]
[Succeeded / Failed / Skipped / Total] 262 / 654 / 35 / 951:  95%|█████████▌| 952/1000 [16:21:08<49:28, 61.84s/it]
[Succeeded / Failed / Skipped / Total] 262 / 655 / 35 / 952:  95%|█████████▌| 952/1000 [16:21:08<49:28, 61.84s/it]
[Succeeded / Failed / Skipped / Total] 262 / 655 / 35 / 952:  95%|█████████▌| 953/1000 [16:22:08<48:26, 61.83s/it]
[Succeeded / Failed / Skipped / Total] 262 / 656 / 35 / 953:  95%|█████████▌| 953/1000 [16:22:08<48:26, 61.83s/it]
[Succeeded / Failed / Skipped / Total] 262 / 656 / 35 / 953:  95%|█████████▌| 954/1000 [16:23:03<47:24, 61.83s/it]
[Succeeded / Failed / Skipped / Total] 262 / 657 / 35 / 954:  95%|█████████▌| 954/1000 [16:23:03<47:24, 61.83s/it]
[Succeeded / Failed / Skipped / Total] 262 / 657 / 35 / 954:  96%|█████████▌| 9


[Succeeded / Failed / Skipped / Total] 267 / 672 / 36 / 975:  98%|█████████▊| 976/1000 [16:45:12<24:43, 61.80s/it]
[Succeeded / Failed / Skipped / Total] 267 / 673 / 36 / 976:  98%|█████████▊| 976/1000 [16:45:12<24:43, 61.80s/it]
[Succeeded / Failed / Skipped / Total] 267 / 673 / 36 / 976:  98%|█████████▊| 977/1000 [16:46:28<23:41, 61.81s/it]
[Succeeded / Failed / Skipped / Total] 267 / 674 / 36 / 977:  98%|█████████▊| 977/1000 [16:46:28<23:41, 61.81s/it]
[Succeeded / Failed / Skipped / Total] 267 / 674 / 36 / 977:  98%|█████████▊| 978/1000 [16:46:59<22:39, 61.78s/it]
[Succeeded / Failed / Skipped / Total] 268 / 674 / 36 / 978:  98%|█████████▊| 978/1000 [16:46:59<22:39, 61.78s/it]
[Succeeded / Failed / Skipped / Total] 268 / 674 / 36 / 978:  98%|█████████▊| 979/1000 [16:47:00<21:36, 61.72s/it]
[Succeeded / Failed / Skipped / Total] 268 / 674 / 37 / 979:  98%|█████████▊| 979/1000 [16:47:00<21:36, 61.72s/it]
[Succeeded / Failed / Skipped / Total] 268 / 674 / 37 / 979:  98%|█████████▊| 9

[Succeeded / Failed / Skipped / Total] 274 / 689 / 37 / 1000: 100%|██████████| 1000/1000 [17:10:10<00:00, 61.81s/it]



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 274    |
| Number of failed attacks:     | 689    |
| Number of skipped attacks:    | 37     |
| Original accuracy:            | 96.3%  |
| Accuracy under attack:        | 68.9%  |
| Attack success rate:          | 28.45% |
| Average perturbed word %:     | 14.63% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 116.54 |
+-------------------------------+--------+
The above are results for DeepWordBugGao2018_ag-news1000_mp0.3_nv11_logit.
